# 1. 변수설정(.env 파일에 키를 입력하세요.)

In [ ]:
from openai import OpenAI
import pandas as pd
from datasets import Dataset
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
from ragas import evaluate
from datetime import datetime
import langchain_openai
from langchain.embeddings import OpenAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper
import os
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

True

In [4]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# 2. 테스트할 답변생성 코드 작성

In [ ]:
# 테스트할 답변생성 코드 작성


# 3. 테스트 수행

In [ ]:
def answer_generate(prompt, model_name, api_key=None):
    if "gpt" in model_name:
        client = OpenAI(api_key=api_key)
        messages = [{
            "role": "system",
            "content": "You are an assistant that generates helpful responses based on the user's request."
        },
        {
            "role": "user",
            "content": prompt
        }
        ]
        response = client.chat.completions.create(model=model_name, messages=messages, temperature=0)
        answer = response.choices[0].message.content
        return answer
    else:
        raise Exception("gpt 전용 답변생성 함수 입니다.")

# 4. 테스트 수행(context, prompt 변수를 테스트 코드에 맞게 수정합니다.)

In [ ]:
# 생성모델명
model_name = 'gpt-4.1-mini'
ragas_emb_model_name = "text-embedding-3-small"

# 1. eval_set 로딩
df = pd.read_csv("./evaluation_data/eval_set.csv", encoding='utf-8-sig')

# 2. 문서 검색 및 답변 생성
questions, ground_truths, answers, contexts = [], [], [], []

for i, row in df.iterrows():
    query = row['input']
    ground_truth = row['output']

    # 리트리버 결과들의 리스트 타입 데이터를 context 변수로 합니다.
    context = ['리트리버 결과 1', '리트리버 결과 2']

    # query와 리트리버 결과 포함한 텍스트를 prompt 변수로 합니다.
    prompt = query + '리트리버 결과를 포함한 텍스트'

    answer = answer_generate(prompt, model_name, OPENAI_API_KEY)
    
    # 기록
    questions.append(query)
    ground_truths.append(ground_truth)
    answers.append(answer)
    contexts.append(context)

# 3. Ragas 입력 데이터셋 생성
eval_dataset = Dataset.from_dict({
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths,
})

# 4. RAGAS 평가 실행
ragas_llm = langchain_openai.OpenAI(model=model_name, api_key=OPENAI_API_KEY)
openai_emb = OpenAIEmbeddings(model=ragas_emb_model_name, api_key=OPENAI_API_KEY)
ragas_emb = LangchainEmbeddingsWrapper(openai_emb)

result = evaluate(
    eval_dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ],
    llm=ragas_llm,
    embeddings=ragas_emb,
    batch_size=2
)

result_df = result.to_pandas()

# 평균을 구할 특정 칼럼 목록
target_columns = ['faithfulness', 'answer_relevancy', 'context_precision', 'context_recall']

# 평균 행 생성
mean_data = {col: result_df[col].mean() for col in target_columns}

# 나머지 칼럼은 NaN 처리
for col in result_df.columns:
    if col not in mean_data:
        mean_data[col] = None

# 평균 행을 DataFrame에 추가
result_df_with_mean = pd.concat([result_df, pd.DataFrame([mean_data], index=['평균'])])
current_time = datetime.now().strftime("%Y%m%d%H%M%S")
result_df_with_mean.to_csv(f"../../evaluation_data/eval_result_{current_time}.csv", encoding="utf-8-sig")

# 전체점수 평균
average_row = result_df_with_mean[target_columns].loc["평균"]
average_total = sum(average_row) / len(average_row)
print("최종평가 전체점수 항목별 값:", dict(average_row))
print("최종평가 전체점수 평균:", average_total)
print("최종평가 데이터프레임\n")
result_df_with_mean